In [756]:
''' The Jupyter notebook is a web-based notebook environment for interactive computing. '''
print(f"ssl verification via Search-Guard")

ssl verification via Search-Guard


In [757]:
from elasticsearch import Elasticsearch
import os
import json
import pandas as pd
import jsondiff
import logging
from dotenv import load_dotenv
import socket
import requests
import sys
import warnings
warnings.filterwarnings("ignore")

In [758]:
''' pip install python-dotenv'''
load_dotenv() # will search for .env file in local folder and load variables

True

In [759]:
INDEX = "om_test"

In [760]:
ca_cert_path = '../upgrade-script/certs/qa13_new/qa13-es8-ca.pem'

In [761]:
''' https://restfulapi.net/http-status-codes/ '''
http_status_code = {
    200 : 'Indicates that the request has succeeded.',
    201 : 'Indicates that the request has succeeded and a new resource has been created as a result.',
    400 : 'The server could not understand the request due to incorrect syntax. The client should NOT repeat the request without modifications.',
    401 : 'Unauthorized rquests. Insufficient permissions',
    403 : 'Unauthorized request. Insufficient permissions. The client does not have access rights to the content. ',
    500 : 'The server encountered an unexpected condition that prevented it from fulfilling the request.'
}

In [762]:
''' base64 encode '''
def base64_encode_for_search_guard(id_pass):
    ''' format -> <id>:<password> '''
    encoded = '{}'.format(base64.b64encode(id_pass.encode('utf-8')).decode())
    # print(encoded)
    return encoded

In [763]:
''' base64 decode '''
def base64_decode_for_search_guard(id_pass):
    ''' format -> <id>:<password> '''
    return base64.b64decode(id_pass).decode('utf-8')

In [764]:
def get_headers(auth):
    ''' Elasticsearch Header '''
    ''' 
    Basic Authentication is a method for an HTTP user agent (e.g., a web browser) 
    to provide a username and password when making a request. 
    You can send the authorization header 
    when making requests and accessing to ES Cluster based on Search-Guard as X-pack. 
      
    Basic Auth : 
    {
        'Content-type': 'application/json', 
        'Authorization' : 'Basic base64.encode(id:password), 
        'Connection': 'close'
    }
    '''
    return {
            'Content-type': 'application/json', 
            'Authorization' : '{}'.format(os.getenv(auth)),
            'Connection': 'close'
    }

In [765]:
# source_es_host = "http://localhost:9201"
source_es_host = os.getenv('ES_DEV_V8_HOST')

### Verify if user can be accessed to the ES cluster with Search Guard as X-pack
* ES Version : ES v.8.17.0 via https protocal
* X-pack : Search Guard for ES v.8.17.0
* Library : ES cliient (https://elasticsearch-py.readthedocs.io/en/v8.17.0/)
* Pass the link to the certificate for ssl validation with user credential via get_es_instance function

In [766]:
def get_es_instance(auth):
    ''' create es instance with user credential and ssl verification '''
    try:
        ''' To disable certificate verification, at the client side, one can use verify attribute.  -- Verify False''' 
        # es_client = Elasticsearch(hosts="{}".format(source_es_host), headers=get_headers(), timeout=5,  verify_certs=False)
        
        ''' We can also pass the link to the certificate for validation via python requests only. '''
        ''' If we don't pass the link for the certification, we may get the message like 
        caused by: SSLError([SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)) if it doesn't exist ca certs
        '''
        es_client = Elasticsearch(hosts="{}".format(source_es_host), 
                                  use_ssl=True, 
                                  ca_certs=ca_cert_path, 
                                  headers=get_headers(auth), 
                                  timeout=5,  
                                  verify_certs=True
                                 )
        return es_client
    except Exception as e:
        print(str(e))


In [767]:
es_client_wx = get_es_instance('BASIC_AUTH_WX')

In [768]:
try:
    print(es_client_wx.cat.health())
    # print(json.dumps(es_client.cat.indices(params={"format": "json"}), indent=2))
    es_health_dict = es_client_wx.cluster.health()
    print(json.dumps(es_health_dict, indent=2))
    df = pd.DataFrame.from_dict([es_health_dict])
except Exception as e:
    print(str(e))

1740429274 20:34:34 supplychain-logging-qa13-es8 green 3 3 293 251 0 0 0 0 0 - 100.0%

{
  "cluster_name": "supplychain-logging-qa13-es8",
  "status": "green",
  "timed_out": false,
  "number_of_nodes": 3,
  "number_of_data_nodes": 3,
  "active_primary_shards": 251,
  "active_shards": 293,
  "relocating_shards": 0,
  "initializing_shards": 0,
  "unassigned_shards": 0,
  "unassigned_primary_shards": 0,
  "delayed_unassigned_shards": 0,
  "number_of_pending_tasks": 0,
  "number_of_in_flight_fetch": 0,
  "task_max_waiting_in_queue_millis": 0,
  "active_shards_percent_as_number": 100.0
}


In [769]:
es_client_om = get_es_instance('BASIC_AUTH_OM')

In [770]:
try:
    print(es_client_om.cat.health())
    # print(json.dumps(es_client.cat.indices(params={"format": "json"}), indent=2))
    es_health_dict = es_client_om.cluster.health()
    print(json.dumps(es_health_dict, indent=2))
    df = pd.DataFrame.from_dict([es_health_dict])
except Exception as e:
        print(str(e))

1740429275 20:34:35 supplychain-logging-qa13-es8 green 3 3 293 251 0 0 0 0 0 - 100.0%

{
  "cluster_name": "supplychain-logging-qa13-es8",
  "status": "green",
  "timed_out": false,
  "number_of_nodes": 3,
  "number_of_data_nodes": 3,
  "active_primary_shards": 251,
  "active_shards": 293,
  "relocating_shards": 0,
  "initializing_shards": 0,
  "unassigned_shards": 0,
  "unassigned_primary_shards": 0,
  "delayed_unassigned_shards": 0,
  "number_of_pending_tasks": 0,
  "number_of_in_flight_fetch": 0,
  "task_max_waiting_in_queue_millis": 0,
  "active_shards_percent_as_number": 100.0
}


### Verify if user can be accessed to the ES cluster with Search Guard as X-pack
* ES Version : ES v.8.17.0 via https protocal
* X-pack : Search Guard for ES v.8.17.0
* Library : Request Package
* Pass the link to the certificate for ssl validation  with user credential via get_es_instance function

In [771]:
def request_verify_es_health(auth):
    try:
        ''' wxuser '''
        res = requests.get(
            url=os.getenv("ES_DEV_V8_HOST"), 
            headers=get_headers(auth), 
            verify=ca_cert_path)
    
        if not res.status_code == 200:
           print(res.status_code, http_status_code.get(res.status_code, "None"))
           print('\n')
           return
           
        print(json.dumps(res.json(), indent=2))
        print('\n')
    except Exception as e:
        print(str(e))

In [772]:
request_verify_es_health('BASIC_AUTH_WX')

{
  "name": "supplychain-logging-qa13-1",
  "cluster_name": "supplychain-logging-qa13-es8",
  "cluster_uuid": "iiQ1fnsxSz2c0VDmhWcctg",
  "version": {
    "number": "8.17.0",
    "build_flavor": "default",
    "build_type": "tar",
    "build_hash": "2b6a7fed44faa321997703718f07ee0420804b41",
    "build_date": "2024-12-11T12:08:05.663969764Z",
    "build_snapshot": false,
    "lucene_version": "9.12.0",
    "minimum_wire_compatibility_version": "7.17.0",
    "minimum_index_compatibility_version": "7.0.0"
  },
  "tagline": "You Know, for Search"
}




In [773]:
request_verify_es_health('BASIC_AUTH_OM')

{
  "name": "supplychain-logging-qa13-1",
  "cluster_name": "supplychain-logging-qa13-es8",
  "cluster_uuid": "iiQ1fnsxSz2c0VDmhWcctg",
  "version": {
    "number": "8.17.0",
    "build_flavor": "default",
    "build_type": "tar",
    "build_hash": "2b6a7fed44faa321997703718f07ee0420804b41",
    "build_date": "2024-12-11T12:08:05.663969764Z",
    "build_snapshot": false,
    "lucene_version": "9.12.0",
    "minimum_wire_compatibility_version": "7.17.0",
    "minimum_index_compatibility_version": "7.0.0"
  },
  "tagline": "You Know, for Search"
}


